# Iterated GHZ game
To understand the idea behind the code, check the part "Iterated GHZ." on page 6 in [Distributed Quantum Advantage for Local Problems](https://arxiv.org/pdf/2411.03240).

In [1]:
from quantum import measure_and_update_density_matrix, GHZ

class Game:

    def __init__(self, color):
        self.color = color
        # GHZ state is the shared entangled state between the players
        # In this implementation it is a global density matrix which is shared between three players
        self.GHZ_state = GHZ()
        self.input = [1, 0, 0]
    
    def get_input(self):
        return self.input.pop(0)
    
    def update_state(self, input):
        self.GHZ_state, result = measure_and_update_density_matrix(self.GHZ_state, input)
        return result

In [2]:
class Player:

    def __init__(self, games):
        self.games = games
        self.result_bit_string = []
        
    def play(self):
        for c in range(len(self.games)):
            if c == 0:
                # For the first round we distribute 1, 0, 0 as input
                # "The referee throws the ball to the game"
                input = self.games[c].get_input()
                self.result_bit_string.append(input)
            else:
                # For the rest of the rounds we use the result from the previous round as input
                # Players are playing the game
                input = self.result_bit_string[-1]
                result = self.games[c].update_state(input)
                self.result_bit_string.append(result)

In [3]:
def validate_result(bit_strings, number_of_colors):
    for round in range(number_of_colors):
        s = bit_strings[round][0]
        t = bit_strings[round][1]
        u = bit_strings[round][2]
        prev_sum = 0
        for color in range(number_of_colors):
            sum_now = s[color] + t[color] + u[color]
            if color == 0:
                if sum_now != 1:
                    return False
            else:
                if prev_sum == 0:
                    if sum_now % 2 != 0:
                        return False
                elif prev_sum == 2:
                    if sum_now % 2 != 1:
                        return False
            prev_sum = sum_now
    return True            

In [4]:
# Run the game with different number of colors
number_of_colors = 12

# Set up the game and distribute one round of information
players = []
for c in range(number_of_colors):
    games = [Game(color) for color in range(number_of_colors)]
    players_for_color = [Player(games) for _ in range(3)]
    players.append(players_for_color)

# Let the players compute the results relying on the shared entangled states
results = []
for team in players:
    mid_results = []
    for player in team:
        player.play()
        mid_results.append(player.result_bit_string)
    results.append(mid_results)

for r in results:
    print(r)

print("Did players win the iterated GHZ: ", validate_result(results, number_of_colors))

[[1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1]]
[[1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1], [0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1], [0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0]]
[[1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]]
[[1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1], [0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1]]
[[1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1], [0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1]]
[[1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1], [0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0], [0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1]]
[[1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0], [0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0], [0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0]]
[[1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1], [0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1]]
[[1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1], [0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1], [0,